## Tải các thư viện cần thiết 

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn 
# !pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

##  Tổng quan về dữ liệu 

In [ ]:
df = pd.read_csv("/kaggle/input/alzheimers-disease-dataset/alzheimers_disease_data.csv")

In [ ]:
df.head(5)

In [ ]:
df = df.drop(columns=["DoctorInCharge"])

In [ ]:
df.info()

Ta thấy dữ liệu gồm 2149 bản ghi, mỗi bản ghi gồm 35 hàng

In [ ]:
df.describe().T

## Tiền xử lý dữ liệu 

In [ ]:
df.isnull().sum()

Không có dữ liệu nào bị thiếu

In [ ]:
df.duplicated().sum()


Không có hai bản ghi nào trùng lặp

## Trực quan hóa dữ liệu 

In [ ]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
Q1 = df[numeric_columns].quantile(0.25)
Q3 = df[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df[numeric_columns] < (Q1 - 1.5 * IQR)) | (df[numeric_columns] > (Q3 + 1.5 * IQR)))
outliers_count = outliers.sum()
print(f"Số ngoại lệ :\n{outliers_count}")

In [ ]:
axes = df.hist(figsize=(20,15), bins=30, edgecolor='black')
for ax, col in zip(axes.flatten(), numeric_columns):
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Distribution of {col}')
print("Biểu đồ cột cho thấy phân phối dữ liệu tổng quan của các trường: ")
plt.tight_layout()
plt.show()

Từ đây có thể thấy, tập dữ liệu này không quá cân bằng. Số dữ liệu chẩn đoán không mắc bệnh lớn gần gấp đôi số dữ liệu chẩn đoán có bệnh.

In [ ]:
plt.figure(figsize=(14,14))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", linewidths=0.1, annot_kws={"size": 4})  
plt.title("Ma trận tương quan giữa các chỉ số", fontsize=16)  

Từ ma trận tương quan trên ta thấy được mức độ tương quan giữa các thuộc tính là rất yếu!!!

## Sử dụng apriori để tìm những luật kết hợp


In [ ]:
categorical_cols = ['Gender', 
       'Smoking', 
       'FamilyHistoryAlzheimers', 'CardiovascularDisease',
       'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',
       'MemoryComplaints', 'BehavioralProblems', 'Confusion',
       'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
       'Forgetfulness']


Dựa vào các cột dữ liệu phân loại, đi khám phá những đặc trưng của người mắc bệnh.

In [ ]:
def genarate_rules(df, cols, target_col="Diagnosis", min_sup=0.05, min_conf=0.5):
    rules = []
    total = len(df)
    p_y = df[target_col].sum() / total

    for r in range(1, len(cols)+1):
        for th in combinations(cols, r):
            c_th = df[list(th)].all(axis=1)
            sup_x = c_th.sum() / total
            sup_xy = (c_th & (df[target_col] == 1)).sum() / total

            if sup_x == 0:
                continue
            conf = sup_xy / sup_x
            lift = conf / p_y

            if sup_xy >= min_sup and conf >= min_conf:
                rules.append({
                    "rule": th,
                    "sup": round(sup_xy, 3),
                    "conf": round(conf, 3),
                    "lift": round(lift, 3)
                })
    return pd.DataFrame(rules).sort_values(by="lift", ascending=False)
    

In [ ]:
df_subset = df[categorical_cols + ["Diagnosis"]]

rules_df = genarate_rules(df_subset, categorical_cols)


In [ ]:
rules_df

## Chia dữ liệu để chạy mô hình dự đoán 

In [ ]:
x = df.drop(columns=["PatientID", "Diagnosis"])
y = df["Diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [ ]:
x_test

In [ ]:
y_test

## MODEL dự đoán

### Thêm các thư viện cần thiết


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB


best_models = {}

#### Hàm đánh giá giải thuật 

In [ ]:
def evaluate_model(grid):
    best_model = grid.best_estimator_
    print(" Best Parameters:", grid.best_params_)

    y_pred = best_model.predict(x_test)

    print("\n Classification Report:")
    print(classification_report(y_test, y_pred))

    print("\n Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()


### Chúng em sử dụng các thuật toán sau để đánh dự đoán: ...

### Chi tiết các thuật toán

#### SVM

In [ ]:
categorical_cols = ['Gender', 'Smoking', 
    'FamilyHistoryAlzheimers', 'CardiovascularDisease',
    'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',
    'MemoryComplaints', 'BehavioralProblems', 'Confusion',
    'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
    'Forgetfulness']
numerical_cols = [col for col in x.columns if col not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [ ]:
model_pipeline_svc = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', SVC())
])

param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__gamma': ['scale', 'auto'],
    'classifier__kernel': ['rbf', 'linear']
}
grid_search = GridSearchCV(model_pipeline_svc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_
evaluate_model(grid_search)

SVM không phù hợp có thể là do có quá nhiều dữ liệu phân loại, khi one hot làm tăng số chiều và rời rạc dữ liệu .Cũng có thể do dữ liệu phi tuyến. SVM dựa trên khoảng cách, việc onehot có thể làm các dữ liệu mất đi ý nghĩa về khoảng cách. Hiểu đơn giản là khi tăng số chiều thì làm tăng khoảng cách, làm ảnh hưởng đến các đặc trưng khác.

#### Naive Bayes

In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', GaussianNB())
])

param_grid = {
    'classifier__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

evaluate_model(grid_search)

#### DecisionTree 

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

param_grid = {
    'criterion': ['entropy'], 
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2'],
    'class_weight': [None, 'balanced']
}


dt_grid = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
best_models['DecisionTree'] = dt_grid

dt_grid.fit(x_train, y_train)

best_model = dt_grid.best_estimator_

evaluate_model(dt_grid)

#### RandomForest

In [ ]:
rf_model = RandomForestClassifier(random_state=42)

param_grid_rf = {
    'n_estimators': [50, 100],                      
    'criterion': ['entropy'],      
    'max_depth': [None, 10, 20, 30],                     
    'min_samples_split': [2, 5, 10],                    
    'min_samples_leaf': [1, 2, 4],          
    'bootstrap': [True, False],                          
    'class_weight': [None, 'balanced']              
}

grid_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
best_models['RandomForest'] = grid_rf

grid_rf.fit(x_train, y_train)

evaluate_model(grid_rf)

#### XGBoost

In [ ]:
xgb_model = XGBClassifier(
    tree_method='hist',  
    gpu_id=1, 
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_lambda': [0.01, 0.1, 1],
    'reg_alpha': [0, 0.1, 0.5]
}

random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid_xgb,
    n_iter=100,
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
best_models['XGBoost'] = random_search_xgb

random_search_xgb.fit(x_train, y_train)

evaluate_model(random_search_xgb)

#### LightGBM

In [ ]:
lgbm_model = LGBMClassifier(
    device='gpu',     
    boosting_type='gbdt',
    random_state=42
)

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10, -1],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_lambda': [0.01, 0.1, 1],
    'reg_alpha': [0, 0.1, 0.5]
}

random_search_lgbm = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=param_grid_lgbm,
    n_iter=100,
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
best_models['LightGBM'] = random_search_lgbm


random_search_lgbm.fit(x_train, y_train)

evaluate_model(random_search_lgbm)

#### CatBoost

In [ ]:
catboost_model = CatBoostClassifier(
    task_type='GPU',          
    devices='0',              
    verbose=0,                
    random_state=42
)

# Grid tham số phù hợp với CatBoost
param_grid_catboost = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bagging_temperature': [0, 1, 2, 5],
    'border_count': [32, 64, 128],  
    'random_strength': [1, 5, 10]
}

random_search_catboost = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=param_grid_catboost,
    n_iter=100,            
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=1
)
best_models['CatBoost'] = random_search_catboost

random_search_catboost.fit(x_train, y_train)

evaluate_model(random_search_catboost)

In [ ]:
# # Tạo các pipeline cho các thuật toán
# models = {
#     "LogisticRegression": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', LogisticRegression(max_iter=1000))
#     ]),
#     "KNN": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', KNeighborsClassifier())
#     ]),
#     "SVM": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', SVC())
#     ]),
#     "DecisionTree": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', DecisionTreeClassifier())
#     ]),
#     "RandomForest": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', RandomForestClassifier())
#     ]),
#     "NaiveBayes": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', GaussianNB())
#     ]),
#     "XGBoost": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
#     ]),
#     "LightGBM": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', LGBMClassifier())
#     ]),
#     "CatBoost": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', CatBoostClassifier(verbose=0))
#     ]),
# }

# # Grid Search cho các thuật toán có thể tinh chỉnh
# param_grids = {
#     "LogisticRegression": {
#         'classifier__C': [0.1, 1.0, 10],
#         'classifier__solver': ['lbfgs', 'liblinear']
#     },
#     "KNN": {
#         'classifier__n_neighbors': [3, 5, 7]
#     },
#     "SVM": {
#         'classifier__C': [0.1, 1, 10],
#         'classifier__kernel': ['linear', 'rbf']
#     },
#     "DecisionTree": {
#         'classifier__criterion': ['entropy'],
#         'classifier__max_depth': [None, 10, 20],
#         'classifier__min_samples_split': [2, 5]
#     },
#     "RandomForest": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__max_depth': [None, 10]
#     },
#     "XGBoost": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__learning_rate': [0.05, 0.1]
#     },
#     "LightGBM": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__learning_rate': [0.05, 0.1]
#     },
#     "CatBoost": {
#         'classifier__iterations': [100, 200],
#         'classifier__learning_rate': [0.05, 0.1]
#     }
# }

# # Huấn luyện từng mô hình
# for model_name, model in models.items():
#     print(f"\nTraining {model_name}...")

#     if model_name in param_grids:
#         grid = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1)
#         grid.fit(x_train, y_train)
#         best_models[model_name] = grid.best_estimator_
#         print(f"Best {model_name} Parameters: {grid.best_params_}")
#     else:
#         model.fit(x_train, y_train)
#         best_models[model_name] = model

#     # Dự đoán và đánh giá
#     y_pred = best_models[model_name].predict(x_test)
#     print(f"\n{model_name} Classification Report:")
#     print(classification_report(y_test, y_pred))

## Đánh giá 

In [ ]:
from sklearn.metrics import confusion_matrix
fig, axes = plt.subplots(3, 3, figsize=(18, 18)) 
axes = axes.flatten()  
model_names = list(best_models.keys())

for i, model_name in enumerate(model_names):
    y_pred = best_models[model_name].predict(x_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f"{model_name} Confusion Matrix")
    axes[i].set_xlabel("Predicted")
    axes[i].set_ylabel("Actual")

for j in range(len(model_names), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

for model_name in model_names:
    model = best_models[model_name]

    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(x_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{model_name} (AUC = {roc_auc:.2f})")
    else:
        print(f"Bỏ qua {model_name} vì không hỗ trợ predict_proba.")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Different Models")
plt.legend()
plt.show()


## Khám phá dữ liệu 

In [ ]:
!pip install hdbscan

In [ ]:
import hdbscan

model = hdbscan.HDBSCAN(min_cluster_size=2)
hdb_labels = model.fit_predict(x)
print(len(set(hdb_labels)))
print(set(hdb_labels))
df['hdb_labels'] = hdb_labels

# for i in range(0, 10):
#     print(f"Bệnh nhân {x.iloc[i]} - Cụm: {hdb_labels[i]}")


In [ ]:
df.head(100)

In [ ]:
c_10 = 0
c_11 = 0
c_00 = 0
c_01 = 0
c = 0
for i in range(2148):
    if df.iloc[i]["Diagnosis"] == 1 and hdb_labels[i] == 0:
        c_10 += 1
    elif df.iloc[i]["Diagnosis"] == 1 and hdb_labels[i] == 1:
        c_11 += 1
    elif df.iloc[i]["Diagnosis"] == 0 and hdb_labels[i] == 0:
        c_00 += 1
    elif df.iloc[i]["Diagnosis"] == 0 and hdb_labels[i] == 1:
        c_01 += 1
    else:
        c += 1

print(c_10)
print(c_11)
print(c_00)
print(c_01)
print(c)

In [ ]:
d = [0,0,0]
for i in range(2148):
    if hdb_labels[i] == 0:
        d[0] += 1
    elif hdb_labels[i] == 1:
        d[1] += 1
    else:
        d[2] += 1
print(d)

In [ ]:
df_filtered = df[df["hdb_labels"] == 1]
df_filtered

In [ ]:
xx = df_filtered.drop(columns=["PatientID", "Diagnosis", "hdb_labels"])
yy = df_filtered["Diagnosis"]

In [ ]:
xx_train, xx_test, yy_train, yy_test = train_test_split(xx,yy,test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Hàm đánh giá mô hình
def evaluate_model(grid):
    best_model = grid.best_estimator_
    print("Best Parameters:", grid.best_params_)

    yy_pred = best_model.predict(xx_test)

    print("\nClassification Report:")
    print(classification_report(yy_test, yy_pred))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(yy_test, yy_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

# Khởi tạo mô hình Random Forest
rf_model1 = RandomForestClassifier(random_state=42)

# Thiết lập grid search
param_grid_rf1 = {
    'n_estimators': [50, 100],
    'criterion': ['entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced']
}

# Grid Search với cross-validation
grid_rf1 = GridSearchCV(
    estimator=rf_model1,
    param_grid=param_grid_rf1,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Huấn luyện
grid_rf1.fit(xx_train, yy_train)

# (Tùy chọn) Lưu mô hình tốt nhất vào dictionary
best_models = {}
best_models['RandomForest'] = grid_rf1

# Đánh giá mô hình
evaluate_model(grid_rf1)


In [ ]:
!pip install kmodes

In [ ]:
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

costs = []
K_range = range(1, 10)
data = np.array(x)
categorical_columns = [1,2,3,5,10,11,12,13,14,15,24,25,27,28,29,30,31]

all_columns = list(range(data.shape[1]))
numerical_columns = [i for i in all_columns if i not in categorical_columns]

numerical_data = data[:, numerical_columns].astype(float)
categorical_data = data[:, categorical_columns]

scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(numerical_data)

final_data = np.empty_like(data, dtype=object)

for i, col in enumerate(numerical_columns):
    final_data[:, col] = numerical_scaled[:, i]

for i, col in enumerate(categorical_columns):
    final_data[:, col] = categorical_data[:, i]


for k in K_range:
    kproto = KPrototypes(n_clusters=k, init='Huang', random_state=42)
    kproto.fit_predict(final_data, categorical=categorical_columns)  
    costs.append(kproto.cost_)

plt.plot(K_range, costs, marker='o')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Cost (Inertia)')
plt.title('Elbow Method for K-Prototypes')
plt.show()


In [ ]:
kproto = KPrototypes(n_clusters=3, init='Huang', random_state=42)
labels = kproto.fit_predict(final_data, categorical=categorical_columns)

In [ ]:
centroids = kproto.cluster_centroids_
print("Tâm cụm:")
print(centroids)

In [ ]:
df = pd.DataFrame(final_data)
df['cluster'] = labels

print("Số điểm trong mỗi cụm:")
print(df['cluster'].value_counts().sort_index())

In [ ]:
df

In [ ]:
y
print(len(y))

In [ ]:
c1 = 0
c2 = 0
c0 = 0
for i in range(len(y)):
    if df.iloc[i]["cluster"] == 0 and y[i] == 1:
        c0 += 1
    elif df.iloc[i]["cluster"] == 1 and y[i] == 1:
        c1 += 1
    elif df.iloc[i]["cluster"] == 2 and y[i] == 1:
        c2 += 1
print(f"Số người mắc bệnh ở cụm 0 là: {c0}")
print(f"Số người mắc bệnh ở cụm 1 là: {c1}")
print(f"Số người mắc bệnh ở cụm 2 là: {c2}")

Không có gì đặc biệt tại các cụm được phân, không có cụm nào chiếm đa số các bệnh nhân. 